In [1]:
from torch import optim
from lib.datasets import get_stock_price,train_test_split
from lib.aug import apply_augmentations,parse_augmentations
import torch
from torch import nn
import torch.nn.functional as F
from typing import List
from lib.utils import sample_indices,compute_mmd


In [2]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
    ],
    "device" : "cuda:0",
    "depth" : 4,
}

In [3]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))
print("Before augmentation shape:",x_real_train.shape)
if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_sig = apply_augmentations(x_real_train,sig_config["augmentations"])
    # Input dimension of encoder
    # We'll flat the tensor
    input_dim = x_aug_sig.shape[1]*x_aug_sig.shape[2]
print("After augmentation shape:",x_aug_sig.shape)
x_aug_sig = x_aug_sig.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
Before augmentation shape: torch.Size([985, 20, 1])
torch.Size([985, 20, 2])
torch.Size([985, 39, 4])
After augmentation shape: torch.Size([985, 39, 4])


In [4]:
# from models.infovae import InfoVAE, InfoVAE_train
# # Training loop
# vae = InfoVAE(x_aug_sig, epoch=100, batch_size=128, hidden_dims=[156, 64, 32], device='cuda')
# optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

# for epoch in range(vae.epoch):
#     # Sample batch
#     indices = torch.randperm(x_aug_sig.size(0))[:vae.batch_size]
#     batch = x_aug_sig[indices].to('cuda')  # [128, 39, 4]
    
#     # Forward pass
#     mean, log_var, z = vae.encode(batch)
#     reconstructed = vae.decode(z)
    
#     # Compute loss
#     total_loss= vae.loss(mean, log_var, batch, reconstructed, lambda_mmd=10)
    
#     # Backpropagation
#     optimizer.zero_grad()
#     total_loss.backward()
#     optimizer.step()

In [ ]:
# from models.wae import WAE,WAE_train

# # Model hyperparameters
# lr = 1e-3  # Learning rate
# batch_size = 128
# epoch = 20001
# latent_dim = 20  # Latent dimension
# hidden_dims = [156, 100, latent_dim]  # Hidden dimensions

# wae = WAE(
#     x_aug_sig=x_aug_sig,
#     epoch=epoch,
#     batch_size=batch_size,
#     hidden_dims=hidden_dims,
#     latent_dim=latent_dim,
#     device='cuda'
# )
# optimizer = torch.optim.Adam(wae.parameters(), lr=lr)

# # Train the model
# WAE_train(wae, optimizer)

Epoch 0 loss 9265150.0
Epoch 500 loss 3045.32958984375
Epoch 1000 loss 3797.83837890625
Epoch 1500 loss 2912.18017578125
Epoch 2000 loss 3642.317138671875
